In [ ]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
import sys,cv2
sys.path.append('../')
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

from Utils.utils import *
from Utils.f1_score import *
from ipywidgets import interact
%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

from jupyterthemes import jtplot
# set "context" (paper, notebook, talk, poster)
jtplot.style(theme='grade3',context='talk', fscale=2.5, spines=True, gridlines='-',ticks=True, grid=True, figsize=(6, 4.5))
plotcolor = (0, 0.6, 1.0)

import torch
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn

import deepdish as dd

import timeit,gc

data_folder = 'D:/data/HPA/all/'
model_folder = 'D:/data/HPA/models/'
USE_SMALL_IMAGES = True
USE_ONLY_POI = True

print("Using GPU:",torch.cuda.is_available())
print("Using device ",torch.cuda.get_device_name(0))
print("Done.")

### Load test data

In [ ]:
%%time
if USE_ONLY_POI:
    filename = 'test_poi'
else:
    filename = 'test_all_channel_'
    
if USE_SMALL_IMAGES:
    d = dd.io.load(data_folder+filename+'_small.h5')
else:
    d = dd.io.load(data_folder+filename+'.h5')

    
X = d['X'].astype(np.float32) / 255.0 # torch likes float images

y = d['labels']
   
print("Shapes are:")
print(X.shape)
print(y.shape)
print("Done")

### Load model

In [ ]:
#Setup resnet model
net = models.resnet18(pretrained=False)
modelname = "baseline"

if USE_ONLY_POI:
    net.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3,bias=False)
else:
    net.conv1 = nn.Conv2d(4, 64, kernel_size=7, stride=2, padding=3,bias=False)
    
if USE_SMALL_IMAGES:
    net.fc = nn.Linear(512, 28) #adapt last layer to allow larger input images
else:
    net.fc = nn.Linear(51200, 28) #adapt last layer to allow larger input images

if USE_ONLY_POI:
    modelname = modelname + "_all"
    
if USE_SMALL_IMAGES:
    modelname = modelname + "_small"

net.load_state_dict(torch.load(model_folder+modelname+".model")) 
    
print("Done.")

### Run test set (TODO remember output)

In [ ]:
# Run validation
gc.collect() 
net.eval()

batch_size = 67
iterations_per_epoch = round(X.shape[0] / batch_size)
time_per_epoch,runtime,running_loss,running_f1 = 0,0,0,0

for i in range(iterations_per_epoch):
    start = timeit.default_timer() #measure time

    X_batch = torch.tensor(X[i:i+batch_size].transpose(0,3,1,2))
    y_batch = torch.tensor(y[i:i+batch_size].astype(np.float32),dtype=torch.float)

    # forward + backward + optimize
    outputs = net(X_batch)

    #compute F1 scores
    act = torch.sigmoid(outputs)

    label = y_batch.detach().numpy().astype(np.bool)
    logits = act.detach().numpy() > 0.5
    print("Targets in batch = ",np.sum(label),"Predicted targets = ",np.sum(logits))
    running_f1 += f1_score(label,logits)

    #measure runtime
    stop = timeit.default_timer()
    time_per_epoch = 0.5 * time_per_epoch + 0.5 * (stop-start) * iterations_per_epoch
    runtime += (stop-start)
    #print performance metrics
    N = ((i+1)*batch_size)
    print('[iteration = (%3d/%d), time = %3ds, est. time per epoch = %5ds] \t ## F1 = %.5f'\
          %(i + 1, iterations_per_epoch, runtime, time_per_epoch, running_f1 / (i+1)))

### Create submission.csv (TODO)